In [ ]:
DATE = "0823_stronger_aug"
DATA_TO_TEST = 'normal' # 'abn1' or 'abn2' or 'normal'
MODEL_EPOCH = '_25ep' ###### 아래 스냅샷 파일과 일치하는지 반드시 확인!!!!!!! ######

#SCORE_VALUE = 0.1
#PRINT_BENIGN_DETECTION = False

import os
import pandas as pd

result_save_base_path = '/home/huray/workspace/han_work/han_test/results/'
result_save_path = os.path.join(result_save_base_path, DATE, DATA_TO_TEST + MODEL_EPOCH)
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
# from keras_retinanet.preprocessing.coco import CocoGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

import matplotlib.pyplot as plt
import cv2

import numpy as np
import time

import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

normal_img_path = []
normal_x1 = []
normal_y1 = []
normal_x2 = []
normal_y2 = []
normal_class_name = []

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

## Load RetinaNet model

In [ ]:
model = keras.models.load_model('snapshots/0620_stronger_aug/resnet101_csv_25.h5', custom_objects=custom_objects)

## Initialize data generators

In [ ]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

if DATA_TO_TEST == 'abn1':
    csv_path = '/home/huray/data/NCC/img_retinanet/data_abn1.csv'
elif DATA_TO_TEST == 'abn2':
    csv_path = '/home/huray/data/NCC/img_retinanet/data_abn2.csv'
elif DATA_TO_TEST == 'normal':
    csv_path = 'data_normal_merged.csv'
else:
    print('WRONG DATA CSV PATH!')
    raise

# create a generator for testing data
val_generator = CSVGenerator(
    csv_path,
    '/home/huray/data/NCC/img_retinanet/class_2_onlyM.csv',
    val_image_data_generator,
    batch_size=1
)

test_file_df = pd.read_csv(csv_path, header=None)
num_of_test_files = len(test_file_df)
print(num_of_test_files)

## Run detection

In [ ]:
img_index_list = list(range(0, num_of_test_files))
img_index_list_temp = img_index_list.copy()
print(len(img_index_list))

In [ ]:
for index in img_index_list:
    pass_this_file = False
    
    # load image
    image, image_path = val_generator.load_image(index, get_image_path=True)
    print(image_path)

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = val_generator.preprocess_image(image)
    image, scale = val_generator.resize_image(image)
    

    # process image
#     start = time.time()
    _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))
#     print("processing time: ", time.time() - start)

    # compute predicted labels and scores
    #predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
    #scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

    # correct for image scale
    detections[0, :, :4] /= scale
      
    # 이미지 하나에 annotation 여러개 있는 경우 있으므로 리스트로 저장해 둠.
    x1_tmp = []
    y1_tmp = []
    x2_tmp = []
    y2_tmp = []
    cls_tmp = []
    
    # save detected object annotations
    number_of_detected_annotation = 0
    
    #for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
    for idx in enumerate(detections) :
    #    if score < SCORE_VALUE:
    #        continue
            
        b = detections[0, idx, :4].astype(int)

        x1_tmp.append(b[0])
        y1_tmp.append(b[1])
        x2_tmp.append(b[2])
        y2_tmp.append(b[3])
        cls_tmp.append('M')
            
        number_of_detected_annotation += 1
    
        #cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 5)
            
    for i_index in range(number_of_detected_annotation):
        mammo_jpg_path = os.path.join(result_save_path, image_path.split('/')[-1].split('.')[0]+'_'+str(i_index)+'.jpg')
        #cv2.imwrite(os.path.join(result_save_path, image_path.split('/')[-1].split('.')[0]+'_'+str(i_index)+'.jpg'), draw)
        cv2.imwrite(mammo_jpg_path, draw)
        
        normal_img_path.append(mammo_jpg_path)
        normal_x1.append(x1_tmp[i_index])
        normal_y1.append(y1_tmp[i_index])
        normal_x2.append(x2_tmp[i_index])
        normal_y2.append(y2_tmp[i_index])
        normal_class_name.append(cls_tmp[i_index])

In [ ]:
normal_data_df = pd.DataFrame({'img_path':normal_img_path, 'x1':normal_x1, 'y1':normal_y1, 'x2':normal_x2, 'y2':normal_y2, 'class_name':normal_class_name})
normal_data_df = normal_data_df[['img_path', 'x1', 'y1', 'x2', 'y2', 'class_name']]

In [ ]:
normal_data_df.to_csv('/home/huray/workspace/han_work/han_test/data_normal_annotation.csv', header=False, index=False)

In [ ]:
print("DONE!" + DATA_TO_TEST + MODEL_EPOCH)